In [1]:
import pandas as pd

#### NBA api did not have a clean method to get the basic stats that are readily available on the basketball reference api. This led to some issues as the names of the foreign players do not match exactly with basketball reference having the letters with accents while the nba api doesnt but its worth losing some data points for the data i get from the basketball reference api.

In [8]:
# getting an idea of what the columns i am getting are

pd.read_html('https://www.basketball-reference.com/leagues/NBA_2020_per_game.html')[0].drop_duplicates(keep = 'first').columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [7]:
#getting a list of the seasons to iterate through

seasons = ['2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013',
           '2012', '2011', '2010', '2009', '2008', '2007', '2006',
           '2005', '2004', '2003', '2002', '2001', '2000']

In [23]:
# creating an empty list to store all the dataframes for each year

player_bio_list = []

In [26]:
# iterating through each season and getting the full list of players for each game in that season, 
# also dropping all the unnecessary columns 

for season in seasons:
    player_bios = pd.read_html('https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'.format(season))[0].drop_duplicates(keep = 'first')
    player_bios.drop(columns = ['Rk', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB'], inplace = True)
    player_bio_list.append(player_bios)

In [27]:
# concating all the dataframes in the list to a single dataframe

player_bio = pd.concat(player_bio_list)

In [36]:
# dropping every 20 rows which shows the column names again (from the website)
player_bio = player_bio[player_bio['Player'] != 'Player']

In [37]:
# changing all numberic values in the list from strings to ints/floats

player_bio = player_bio.apply(pd.to_numeric, errors = 'ignore')

In [110]:
# aggregating all the numeric valuesby the mean

player_bio_agg = player_bio.groupby('Player').mean()

In [111]:
# function to find the most common item in a list
# this is for players who are labeled as different positions over the course of their careers, i am only taking the position 
# they were referred to the most out of any of them

def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

In [112]:
# aggregating all the positions into a single string seperated by commas

player_bio_pos = (player_bio.groupby('Player')['Pos'].apply(lambda x: ', '.join(x))).to_frame()

In [113]:
# looping through all the positions, turning them into a list with .split then using the function to find the most frequent

player_bio_pos['Pos'] = [most_frequent(x.split(', ')) for x in player_bio_pos['Pos']]

In [114]:
# concating the 2 datframes

player_bios = pd.concat([player_bio_pos, player_bio_agg], axis = 1)

In [109]:
# to csv

player_bios.to_csv('csv_files/Player_bios.csv')